In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/net/mulan/home/huiwann/spatial/SVC/')
import tifffile
from model.train import *
from model.utils import *
import torch 
from torch.utils.data import DataLoader
import numpy as np
import random

In [2]:
data_path = '/net/mulan/home/huiwann/spatial/SVC/' 
dataset = 'data/seqfish' 
ckpt_folder = f'{data_path}checkpoints/'
device = 'cuda:3'

seed = 2025
torch.manual_seed(seed) 
random.seed(seed) 
np.random.seed(seed)

In [3]:
train_seqfish = np.load(f"{data_path}{dataset}/train_seqfish.npz")  # 或 "cuda"
train_image = train_seqfish["data_ori"]
train_cell_morphology = train_seqfish["cell_morphology"]
train_nuclear_morphology = train_seqfish["nuclear_morphology"]
train_data_location = train_seqfish["location"]
train_cell_cycle_label = train_seqfish["identity_label"]

train_dataset = SVC_Dataset(
    data_ori=train_image,
    location=train_data_location,
    cell_morphology_vec=train_cell_morphology,
    nuclear_morphology_vec=train_nuclear_morphology,
    identity_vec=train_cell_cycle_label,
)
print("number of training cells:", len(train_dataset),', number of genes:', train_image.shape[1])
cell_median_train = np.median(train_image.sum((1,2,3)))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

number of training cells: 157 , number of genes: 1000


In [4]:
read_dir =f'{data_path}{dataset}/gene2vec_weight_seqfish.npy'
gene2vec_weight = torch.from_numpy(np.load(read_dir)).float() ##n_gene * 200
print("shape of gene2vec_weight", gene2vec_weight.shape)
model = SVC(
    gene2vec_weight = gene2vec_weight,
    cell_identity_dim = train_cell_cycle_label.shape[1],
).to(device)


shape of gene2vec_weight torch.Size([1000, 200])


In [5]:
epoch_losses = train_SVC(
    model=model,
    train_loader=train_loader,
    cell_median_train = cell_median_train,
    device=device,
    num_epochs=400,
    ckpt_dir=ckpt_folder,
    ckpt_name="SVC_seqfish",
    early_stop_patience = 50
)


Training:   0%|          | 0/400 [00:00<?, ?epoch/s]

Early stopping at epoch 199 with best loss 37.6105
Saving model checkpoint to /net/mulan/home/huiwann/spatial/SVC/checkpoints/SVC_seqfish.pth
Finished training
